# Data Mining -  Computer Assignment 1
Mohammad Saadati - 
_810198410_

## Introduction
The goal of this computer assignment is review the data set of COVID-19, pre-processing, illustration and express our analysis based on the illustrations performed. This data set have information about patients, new tests, patient deaths and some features related to countries are given separately by day.

## Import Libraries
First of all, we import necessary libraries to use their functions.

In [33]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import math
from scipy.stats import norm
from shapely.geometry import LineString
import jdatetime
import datetime

## Preprocessing
First we load csv file as a DataFrame using pandas library.

In [34]:
df = pd.read_csv("CA1_Dataset.csv")
df

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality,excess_mortality_cumulative_per_million
0,AFG,Asia,Afghanistan,2020-02-24,5.0,5.0,NaN,NaN,NaN,NaN,...,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN
1,AFG,Asia,Afghanistan,2020-02-25,5.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN
2,AFG,Asia,Afghanistan,2020-02-26,5.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN
3,AFG,Asia,Afghanistan,2020-02-27,5.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN
4,AFG,Asia,Afghanistan,2020-02-28,5.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
165631,ZWE,Africa,Zimbabwe,2022-02-26,235803.0,336.0,368.429,5393.0,1.0,1.000,...,1.6,30.7,36.791,1.7,61.49,0.571,NaN,NaN,NaN,NaN
165632,ZWE,Africa,Zimbabwe,2022-02-27,235803.0,0.0,350.143,5393.0,0.0,1.000,...,1.6,30.7,36.791,1.7,61.49,0.571,NaN,NaN,NaN,NaN
165633,ZWE,Africa,Zimbabwe,2022-02-28,236380.0,577.0,401.286,5395.0,2.0,1.286,...,1.6,30.7,36.791,1.7,61.49,0.571,NaN,NaN,NaN,NaN
165634,ZWE,Africa,Zimbabwe,2022-03-01,236871.0,491.0,413.000,5395.0,0.0,1.000,...,1.6,30.7,36.791,1.7,61.49,0.571,NaN,NaN,NaN,NaN


### Question 1
In this part first we use two Pandas function (isna and sum) to count number of missing values at each column.

In [35]:
df.info()
pd.set_option('display.max_rows',None)
df.isna().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 165636 entries, 0 to 165635
Data columns (total 67 columns):
 #   Column                                      Non-Null Count   Dtype  
---  ------                                      --------------   -----  
 0   iso_code                                    165636 non-null  object 
 1   continent                                   155719 non-null  object 
 2   location                                    165636 non-null  object 
 3   date                                        165636 non-null  object 
 4   total_cases                                 162606 non-null  float64
 5   new_cases                                   162464 non-null  float64
 6   new_cases_smoothed                          160480 non-null  float64
 7   total_deaths                                144793 non-null  float64
 8   new_deaths                                  144833 non-null  float64
 9   new_deaths_smoothed                         142734 non-null  float64
 

iso_code                                           0
continent                                       9917
location                                           0
date                                               0
total_cases                                     3030
new_cases                                       3172
new_cases_smoothed                              5156
total_deaths                                   20843
new_deaths                                     20803
new_deaths_smoothed                            22902
total_cases_per_million                         3785
new_cases_per_million                           3927
new_cases_smoothed_per_million                  5905
total_deaths_per_million                       21585
new_deaths_per_million                         21545
new_deaths_smoothed_per_million                23638
reproduction_rate                              40569
icu_patients                                  142246
icu_patients_per_million                      

Now use fillna function to fill out the missing values in the given column.

In [38]:
unique_continent = df['continent'].unique()

for column in df:
    if df[column].isna().sum() > 0:
        if df[column].dtypes != object:
            if df[column].isna().sum() / df.shape[0] > 0.85:
                df[column].fillna(-np.inf, inplace=True)
            else:
                df[column].fillna(df[column].mean(), inplace=True)
        elif column == "continent":
            df['continent'] = df.apply(lambda row: row['location'] if (pd.isna(row['continent']) and (row['location'] in unique_continent)) 
                                       else ('Undefined' if pd.isna(row['continent']) else row['continent']) , axis=1)
        elif column == "tests_units":
            df['tests_units'].fillna(df['tests_units'].mode()[0], inplace=True) 
#             df['tests_units'] = df.apply( lambda row : df['tests_units'].mode()[0] if (df[df['location'] == row['location']]['tests_units'].isnull().values.all())
#                                         else (df[df['location'] == row['location']]['tests_units'].mode()[0] if pd.isna(row['tests_units'])
#                                         else row['tests_units']), axis = 1)

#df.to_csv("df_p1_q1.csv")
df.isna().sum()

iso_code                                      0
continent                                     0
location                                      0
date                                          0
total_cases                                   0
new_cases                                     0
new_cases_smoothed                            0
total_deaths                                  0
new_deaths                                    0
new_deaths_smoothed                           0
total_cases_per_million                       0
new_cases_per_million                         0
new_cases_smoothed_per_million                0
total_deaths_per_million                      0
new_deaths_per_million                        0
new_deaths_smoothed_per_million               0
reproduction_rate                             0
icu_patients                                  0
icu_patients_per_million                      0
hosp_patients                                 0
hosp_patients_per_million               

#### Advantages and disadvantages of using mean for missing values:
Missing values in your data do not reduce your sample size, as it would be the case with listwise deletion (the default of many statistical software packages, e.g. R, Stata, SAS or SPSS). Since mean imputation replaces all missing values, you can keep your whole database.

Replacing missing data by the mean of nonmissing data causes the population SD to be underestimated, but may also obscure important features of the population from which the data were sampled. Another possible disadvantage with using the mean for missing values is that the reason the values are missing in the first place could be dependent on the missing values themselves. (This is called missing not at random.)

Advantages : Easy to apply - Mean will not change

Disadvantages : Results may not be accurate - For large amount of NaN data this method can increase mod value significantly and cause error in results - The variance will decrease

### Question 2

In [46]:
pd.reset_option('display.max_rows')

df_p1_q2 = df[["location", "new_cases", "new_deaths"]]
df_p1_q2 = df_p1_q2.groupby(by="location").sum()

df_p1_q2_p = df[["location", "population"]]
df_p1_q2_p = df_p1_q2_p.groupby(by="location").max()

df_p1_q2 = df_p1_q2.join(df_p1_q2_p)
df_p1_q2

,new_cases,new_deaths,population
location,,,
Afghanistan,1.970531e+05,1.241206e+04,3.983543e+07
Africa,1.123052e+07,2.486680e+05,1.373486e+09
Albania,4.211434e+05,6.042782e+03,2.872934e+06
Algeria,2.650790e+05,9.583034e+03,4.461663e+07
Andorra,3.824900e+04,3.576042e+03,7.735400e+04
...,...,...,...
Wallis and Futuna,4.540000e+02,2.620857e+04,1.109400e+04
World,4.390117e+08,5.946817e+06,7.874966e+09
Yemen,2.325803e+04,5.560042e+03,3.049064e+07


### Question 3
In this part we use **jdatetime.date.fromgregorian** function from `jdatetime` library to convert gregorian date to shamsi date and store the result in *shamsi_date* column of a new dataframe called **df_p1_q3**.

In [30]:
def convert_gregorian_date_to_shamsi_date(gregorian_date):
    y, m , d = gregorian_date.split("-")
    return str(jdatetime.date.fromgregorian(day=int(d),month=int(m),year=int(y)))

#df_p1_q3 = df.copy()
df['shamsi_date'] = df['date']

for index, row in df.iterrows():
    shamsi_date = convert_gregorian_date_to_shamsi_date(row['shamsi_date'])
    df.at[index,'shamsi_date'] = shamsi_date

#df.to_csv("df_p1_q3.csv")
df

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality,excess_mortality_cumulative_per_million,shamsi_date
0,AFG,Asia,Afghanistan,2020-02-24,5.0,5.0,11492.440366,57400.29315,171.252111,172.811556,...,32.778255,37.746,0.5,64.83,0.511,37613.01782,9.404336,15.967077,972.197816,1398-12-05
1,AFG,Asia,Afghanistan,2020-02-25,5.0,0.0,11492.440366,57400.29315,171.252111,172.811556,...,32.778255,37.746,0.5,64.83,0.511,37613.01782,9.404336,15.967077,972.197816,1398-12-06
2,AFG,Asia,Afghanistan,2020-02-26,5.0,0.0,11492.440366,57400.29315,171.252111,172.811556,...,32.778255,37.746,0.5,64.83,0.511,37613.01782,9.404336,15.967077,972.197816,1398-12-07
3,AFG,Asia,Afghanistan,2020-02-27,5.0,0.0,11492.440366,57400.29315,171.252111,172.811556,...,32.778255,37.746,0.5,64.83,0.511,37613.01782,9.404336,15.967077,972.197816,1398-12-08
4,AFG,Asia,Afghanistan,2020-02-28,5.0,0.0,11492.440366,57400.29315,171.252111,172.811556,...,32.778255,37.746,0.5,64.83,0.511,37613.01782,9.404336,15.967077,972.197816,1398-12-09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
165631,ZWE,Africa,Zimbabwe,2022-02-26,235803.0,336.0,368.429000,5393.00000,1.000000,1.000000,...,30.700000,36.791,1.7,61.49,0.571,37613.01782,9.404336,15.967077,972.197816,1400-12-07
165632,ZWE,Africa,Zimbabwe,2022-02-27,235803.0,0.0,350.143000,5393.00000,0.000000,1.000000,...,30.700000,36.791,1.7,61.49,0.571,37613.01782,9.404336,15.967077,972.197816,1400-12-08
165633,ZWE,Africa,Zimbabwe,2022-02-28,236380.0,577.0,401.286000,5395.00000,2.000000,1.286000,...,30.700000,36.791,1.7,61.49,0.571,37613.01782,9.404336,15.967077,972.197816,1400-12-09
165634,ZWE,Africa,Zimbabwe,2022-03-01,236871.0,491.0,413.000000,5395.00000,0.000000,1.000000,...,30.700000,36.791,1.7,61.49,0.571,37613.01782,9.404336,15.967077,972.197816,1400-12-10


### Question 4
Redundant attributes may be able to be detected by correlation analysis and covariance analysis

In [48]:
df.corr()

,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,total_cases_per_million,new_cases_per_million,new_cases_smoothed_per_million,total_deaths_per_million,...,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality,excess_mortality_cumulative_per_million
total_cases,1.000000,0.806597,0.827678,0.967225,0.724304,0.744763,0.103031,0.031797,0.041708,0.126650,...,-0.001528,-0.005497,0.025735,-0.004164,0.010271,0.029899,0.800792,0.136727,0.031628,0.225535
new_cases,0.806597,1.000000,0.976857,0.733564,0.666245,0.652192,0.096005,0.088054,0.097631,0.093638,...,0.003910,-0.001192,0.020614,0.003064,0.012166,0.028530,0.462151,0.087052,0.089183,0.121159
new_cases_smoothed,0.827678,0.976857,1.000000,0.747942,0.661943,0.671936,0.099350,0.077361,0.100115,0.096034,...,0.003902,-0.001449,0.020900,0.002671,0.012276,0.028863,0.506099,0.069582,0.064482,0.121265
total_deaths,0.967225,0.733564,0.747942,1.000000,0.780125,0.802042,0.072167,0.012108,0.015948,0.145034,...,-0.005402,-0.001592,0.031875,-0.006976,0.013403,0.027753,0.914196,0.304267,0.084374,0.338754
new_deaths,0.724304,0.666245,0.661943,0.780125,1.000000,0.975203,0.016969,0.010628,0.010316,0.070385,...,-0.005513,0.004306,0.034758,-0.003561,0.012716,0.029064,0.467864,0.250554,0.355088,0.146948
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
human_development_index,0.029899,0.028530,0.028863,0.027753,0.029064,0.030304,0.345034,0.159039,0.200978,0.324512,...,0.483810,0.082491,0.447246,0.468142,0.847106,1.000000,-0.102242,-0.347135,-0.290497,-0.231463
excess_mortality_cumulative_absolute,0.800792,0.462151,0.506099,0.914196,0.467864,0.637970,0.168976,0.007658,0.016979,0.372680,...,-0.122511,-0.115914,0.102151,-0.168047,-0.195392,-0.102242,1.000000,0.397512,0.167042,0.445995
excess_mortality_cumulative,0.136727,0.087052,0.069582,0.304267,0.250554,0.216891,0.230031,0.055354,0.038520,0.637618,...,-0.261507,-0.066339,0.203881,-0.252829,-0.253115,-0.347135,0.397512,1.000000,0.551706,0.738794
excess_mortality,0.031628,0.089183,0.064482,0.084374,0.355088,0.290312,0.098497,0.116580,0.113713,0.249940,...,-0.119547,0.052966,0.145492,-0.111678,-0.257628,-0.290497,0.167042,0.551706,1.000000,0.382483


### Question 5

In [31]:
df_iran = df[df["iso_code"] == "IRN"]
# df_p1_q5 = df.iloc[71641:72384]
df_iran

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality,excess_mortality_cumulative_per_million,shamsi_date
71639,IRN,Asia,Iran,2020-02-19,2.0,2.0,11492.440366,2.0,2.0,172.811556,...,21.1,50.790872,1.5,76.68,0.783,37613.01782,9.404336,15.967077,972.197816,1398-11-30
71640,IRN,Asia,Iran,2020-02-20,5.0,3.0,11492.440366,2.0,0.0,172.811556,...,21.1,50.790872,1.5,76.68,0.783,37613.01782,9.404336,15.967077,972.197816,1398-12-01
71641,IRN,Asia,Iran,2020-02-21,18.0,13.0,11492.440366,4.0,2.0,172.811556,...,21.1,50.790872,1.5,76.68,0.783,37613.01782,9.404336,15.967077,972.197816,1398-12-02
71642,IRN,Asia,Iran,2020-02-22,28.0,10.0,11492.440366,5.0,1.0,172.811556,...,21.1,50.790872,1.5,76.68,0.783,37613.01782,9.404336,15.967077,972.197816,1398-12-03
71643,IRN,Asia,Iran,2020-02-23,43.0,15.0,11492.440366,8.0,3.0,172.811556,...,21.1,50.790872,1.5,76.68,0.783,-2658.00000,-4.140000,2.790000,-31.260011,1398-12-04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72377,IRN,Asia,Iran,2022-02-26,7030943.0,7039.0,15065.429000,136390.0,224.0,227.429000,...,21.1,50.790872,1.5,76.68,0.783,37613.01782,9.404336,15.967077,972.197816,1400-12-07
72378,IRN,Asia,Iran,2022-02-27,7040467.0,9524.0,14002.143000,136631.0,241.0,227.286000,...,21.1,50.790872,1.5,76.68,0.783,37613.01782,9.404336,15.967077,972.197816,1400-12-08
72379,IRN,Asia,Iran,2022-02-28,7051429.0,10962.0,12838.143000,136838.0,207.0,223.143000,...,21.1,50.790872,1.5,76.68,0.783,37613.01782,9.404336,15.967077,972.197816,1400-12-09
72380,IRN,Asia,Iran,2022-03-01,7060741.0,9312.0,11015.143000,137064.0,226.0,223.571000,...,21.1,50.790872,1.5,76.68,0.783,37613.01782,9.404336,15.967077,972.197816,1400-12-10


### Question 6

In [32]:
df_iran['shamsi_month'] = df_iran["shamsi_date"]

for index, row in df_iran.iterrows():
    y, m , d = row['shamsi_date'].split("-")
    df_iran.at[index,'shamsi_month'] = m

df_iran

<ipython-input-32-8ab355a04e5a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_iran['shamsi_month'] = df_iran["shamsi_date"]


,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality,excess_mortality_cumulative_per_million,shamsi_date,shamsi_month
71639,IRN,Asia,Iran,2020-02-19,2.0,2.0,11492.440366,2.0,2.0,172.811556,...,50.790872,1.5,76.68,0.783,37613.01782,9.404336,15.967077,972.197816,1398-11-30,11
71640,IRN,Asia,Iran,2020-02-20,5.0,3.0,11492.440366,2.0,0.0,172.811556,...,50.790872,1.5,76.68,0.783,37613.01782,9.404336,15.967077,972.197816,1398-12-01,12
71641,IRN,Asia,Iran,2020-02-21,18.0,13.0,11492.440366,4.0,2.0,172.811556,...,50.790872,1.5,76.68,0.783,37613.01782,9.404336,15.967077,972.197816,1398-12-02,12
71642,IRN,Asia,Iran,2020-02-22,28.0,10.0,11492.440366,5.0,1.0,172.811556,...,50.790872,1.5,76.68,0.783,37613.01782,9.404336,15.967077,972.197816,1398-12-03,12
71643,IRN,Asia,Iran,2020-02-23,43.0,15.0,11492.440366,8.0,3.0,172.811556,...,50.790872,1.5,76.68,0.783,-2658.00000,-4.140000,2.790000,-31.260011,1398-12-04,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72377,IRN,Asia,Iran,2022-02-26,7030943.0,7039.0,15065.429000,136390.0,224.0,227.429000,...,50.790872,1.5,76.68,0.783,37613.01782,9.404336,15.967077,972.197816,1400-12-07,12
72378,IRN,Asia,Iran,2022-02-27,7040467.0,9524.0,14002.143000,136631.0,241.0,227.286000,...,50.790872,1.5,76.68,0.783,37613.01782,9.404336,15.967077,972.197816,1400-12-08,12
72379,IRN,Asia,Iran,2022-02-28,7051429.0,10962.0,12838.143000,136838.0,207.0,223.143000,...,50.790872,1.5,76.68,0.783,37613.01782,9.404336,15.967077,972.197816,1400-12-09,12
72380,IRN,Asia,Iran,2022-03-01,7060741.0,9312.0,11015.143000,137064.0,226.0,223.571000,...,50.790872,1.5,76.68,0.783,37613.01782,9.404336,15.967077,972.197816,1400-12-10,12


### Question 7

In [47]:
df_p1_q7 = df_iran.groupby(by="shamsi_month").sum()
df_p1_q7

,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,total_cases_per_million,new_cases_per_million,new_cases_smoothed_per_million,total_deaths_per_million,...,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality,excess_mortality_cumulative_per_million
shamsi_month,,,,,,,,,,,,,,,,,,,,,
01,63306383.0,556926.0,5.075491e+05,2083272.0,9635.0,8891.429000,744529.061,6549.851,5969.145000,24500.791,...,49.6,1308.2,3149.034063,93.0,4754.16,48.546,2.617803e+06,652.235492,1106.447999,58339.021851
02,83525368.0,573693.0,6.056613e+05,2476878.0,12734.0,13219.142000,982319.015,6747.045,7123.017000,29129.888,...,49.6,1308.2,3149.034063,93.0,4754.16,48.546,2.647645e+06,653.144164,1126.102152,59749.668667
03,97098755.0,365373.0,3.744699e+05,2764726.0,7231.0,7429.715000,1141951.911,4297.050,4404.036000,32515.189,...,49.6,1308.2,3149.034063,93.0,4754.16,48.546,2.833954e+06,674.839828,1097.665075,61410.953922
04,110744780.0,594463.0,5.500337e+05,3015183.0,10089.0,9501.003000,1302439.082,6991.316,6468.789000,35460.744,...,49.6,1308.2,3149.034063,93.0,4754.16,48.546,2.774594e+06,671.269828,1112.185075,60712.839543
05,138256546.0,1129211.0,1.104837e+06,3472214.0,19717.0,18797.147000,1625997.441,13280.344,12993.681000,40835.761,...,49.6,1308.2,3149.034063,93.0,4754.16,48.546,3.044016e+06,707.969828,1372.105075,63881.430970
06,171811588.0,870832.0,9.147470e+05,4143782.0,20255.0,20919.141000,2020629.112,10241.614,10758.092000,48733.884,...,49.6,1308.2,3149.034063,93.0,4754.16,48.546,3.105836e+06,724.279828,1485.365075,64608.486145
07,184752391.0,487165.0,4.971884e+05,4490394.0,13605.0,13732.287000,2172822.362,5729.415,5847.296000,52810.295,...,48.0,1266.0,3047.452319,90.0,4600.80,46.980,3.143634e+06,708.575492,1199.327999,64523.169427
08,199408398.0,515940.0,5.111146e+05,4936276.0,16578.0,16233.570000,2345187.650,6067.829,6011.085000,58054.190,...,48.0,1266.0,3047.452319,90.0,4600.80,46.980,3.476114e+06,749.231155,1260.590922,67903.536555
09,214689319.0,428351.0,4.558521e+05,5400007.0,11940.0,13009.002000,2524902.388,5037.721,5361.153000,63508.006,...,48.0,1266.0,3047.452319,90.0,4600.80,46.980,3.458459e+06,759.521155,1109.690922,67695.900927


## Display data

### Question 1

### Question 2

### Question 3

### Question 4

### Question 5

### Question 6

### Question 7

### Question 8

## Bonus questions

### Question 1

### Question 2